In [ ]:
import os
import cv2
import pandas as pd
import glob
from concurrent.futures import ThreadPoolExecutor, as_completed
from cfg import DOWNLOAD_DIR, SCENE_DIR, KEYFRAME_DIR

In [23]:
# Paths
os.makedirs(KEYFRAME_DIR, exist_ok=True)

In [24]:
# Recursively find all videos
videos = glob.glob(os.path.join(DOWNLOAD_DIR, '**', '*.mp4'))
print(f"[INFO] Found {len(videos)} videos to process.")

[INFO] Found 169 videos to process.


In [ ]:
def process_video(video_path):
    base_name = os.path.splitext(os.path.basename(video_path))[0]
    batchh = base_name.split("_")[0]
    scene_path = os.path.join(SCENE_DIR, f"{batchh}/{base_name}.csv")
    scene_list = pd.read_csv(scene_path)
    
    os.makedirs(os.path.join(KEYFRAME_DIR, batchh, base_name), exist_ok=True)
    
    # Extract keyframes
    cap = cv2.VideoCapture(video_path)
    for i, row in scene_list.iterrows():
        median_frame = int(row['median_frame'])
        
        cap.set(cv2.CAP_PROP_POS_FRAMES, median_frame)
        ret, frame = cap.read()
        if ret:
            keyframe_path = os.path.join(KEYFRAME_DIR, f"{batchh}/{base_name}/{base_name}_{median_frame}.jpg")
            cv2.imwrite(keyframe_path, frame)
        else:
            print(f"[WARNING] Could not read frame {median_frame} in {video_name}")

    cap.release()
    print(f"[DONE] {video_path} processed.")


# Process in parallel
max_workers = 4
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_video, v) for v in videos]
    for _ in tqdm(as_completed(futures), total=len(futures), desc="Processing videos"):
        _.result()

[DONE] /mnt/d/AI Challenge/Data/video/L21/L21_V005.mp4 processed.
[DONE] /mnt/d/AI Challenge/Data/video/L21/L21_V003.mp4 processed.
[DONE] /mnt/d/AI Challenge/Data/video/L21/L21_V002.mp4 processed.
